# Python Intro - Final Exercise


In [1]:
import pyodbc
import pandas as pd
import numpy as np

In [2]:
### In windows, Using a ODBC DNS (predefined connection name)
conn = pyodbc.connect('DSN=COLLEGE;Trusted_Connection=yes;')


In [3]:
### Get the students table
students = pd.read_sql("SELECT * FROM Students;",conn) 
Classrooms = pd.read_sql("SELECT * FROM Classrooms;",conn) 
Courses = pd.read_sql("SELECT * FROM Courses;",conn) 
Departments = pd.read_sql("SELECT * FROM Departments;",conn) 
Teachers = pd.read_sql("SELECT * FROM Teachers;",conn) 

In [4]:
students.head()

,StudentId,FirstName,LastName,Gender
0,162,Elmira,Holston,F
1,154,Sierra,Van Vorst,F
2,28,Mycah,Bingham,F
3,54,Alia,Jones,F
4,31,Chanda,Fillmore,F


In [5]:
Classrooms.head()

,CourseId,StudentId,degree
0,1,202,92.031070
1,1,12,81.175916
2,1,232,82.396587
3,1,123,62.420449
4,1,274,72.569900


In [6]:
Courses.head()

,CourseId,CourseName,DepartmentId,TeacherId
0,1,English Begginers,1,15
1,2,Advanced English,1,15
2,3,Proffesional English,1,15
3,4,Mathematics 101,2,2
4,5,Matematics Advanced,2,13


In [7]:
Departments.head()

,DepartmentId,DepartmentName
0,1,English
1,2,Science
2,3,Arts
3,4,Sport


In [8]:
Teachers.head()

,TeacherId,FirstName,LastName,Gender
0,1,Kaylea,Monzingo,F
1,2,Amanda,Dworkin,F
2,3,Jordyn,Baron,F
3,4,Katherine,Dibari,F
4,5,Heather,Metcalf,F


# Questions
 
 
## Q1. Count the number of students on each department

In [9]:
df_inner_merge_class = pd.merge(Classrooms,Courses, on="CourseId")
df_inner_merge_dep = pd.merge(df_inner_merge_class,Departments, on="DepartmentId")
R1 = df_inner_merge_dep.groupby("DepartmentName").StudentId.nunique().reset_index()
R1.columns = ['Department Name','Student Count']
R1

,Department Name,Student Count
0,Arts,137
1,English,84
2,Science,200
3,Sport,81


## Q2. How many students have each course of the English department and the total number of students in the department?

In [22]:
R2=df_inner_merge_dep[df_inner_merge_dep.DepartmentName =='English'].groupby("CourseName").StudentId.nunique()
R2.columns = ['CourseName','Student Count']
R2

R3 = df_inner_merge_dep[df_inner_merge_dep.DepartmentName =='English'].groupby("DepartmentName").StudentId.nunique()
R3.columns = ['CourseName','Student Count']
R3

R23 = R2.append(R3)
R23.rename(index={'English':'total'}, inplace=True)
R23 = R23.reset_index()
R23.columns = ['CourseName','Student Count']
R23

 

,CourseName,Student Count
0,Advanced English,24
1,English Begginers,32
2,Proffesional English,38
3,total,84


## Q3. How many small (<22 students) and large (22+ students) classrooms are needed for the Science department?

In [11]:
R4=df_inner_merge_dep[df_inner_merge_dep.DepartmentId == 2].groupby("CourseName").StudentId.nunique().reset_index()
R4.columns = ['classroom type','Student Count']
R4

R4.loc[R4['Student Count'] >22, 'classroom type'] = 'BIG CLASS'
R4.loc[R4['Student Count'] <22, 'classroom type'] = 'SMALL CLASS'
R4 = R4.groupby('classroom type')['Student Count'].count().reset_index()
R4.columns = ['classroom type','number of classrooms']
R4


,classroom type,number of classrooms
0,BIG CLASS,10
1,SMALL CLASS,6


## Q4. A feminist student claims that there are more male than female in the College. Justify if the argument is correct

In [12]:
R5=students.groupby("Gender").StudentId.nunique().reset_index()
R5.columns = ['Gender','Student Count']
R5

,Gender,Student Count
0,F,165
1,M,115


## Q5. For which courses the percentage of male/female students is over 70%?

In [13]:
df_inner_merge_stud = pd.merge(df_inner_merge_class, students, on= 'StudentId')
R6=df_inner_merge_stud.groupby(['CourseId','CourseName','Gender'])['StudentId'].count().reset_index()

R6.columns = ['CourseId','CourseName','Gender','count gender']
R6

R7=df_inner_merge_stud.groupby(['CourseId','CourseName'])['StudentId'].count().reset_index()

R7.columns = ['CourseId','CourseName','count']
R7

R67=pd.merge(R6,R7, on=['CourseId','CourseName'])
R67
R67['percent']=(R67['count gender'] / R67['count'])*100
R67 = R67[['CourseId','CourseName','percent']]
R67 = R67[R67['percent'] > 70.0]
R67.rename(index={42:0,52:1}, inplace=True)
R67





,CourseId,CourseName,percent
0,22,Sculpture,70.833333
1,29,Tenis,72.222222


## Q6. For each department, how many students passed with a grades over 80?

In [57]:
df_inner_merge_class = pd.merge(Classrooms,Courses, on="CourseId")
df_inner_merge_dep = pd.merge(df_inner_merge_class,Departments, on="DepartmentId")
df_inner_merge_stud1 = pd.merge(df_inner_merge_dep, students, on= 'StudentId')
R8= df_inner_merge_stud1[df_inner_merge_stud1.degree >80].groupby(["DepartmentName"]).StudentId.nunique().reset_index()


R8.columns = ['Department Name','Student Count80']
R8

R81=pd.merge(R8,R1, on=['Department Name'])
R81

R81
R81['count_percent80']=(R81['Student Count80'] / R81['Student Count'])*100.
R81 = R81[['Department Name','Student Count80','Student Count','count_percent80']]
R81

,Department Name,Student Count80,Student Count,count_percent80
0,Arts,61,137,44.525547
1,English,40,84,47.619048
2,Science,122,200,61.000000
3,Sport,39,81,48.148148


## Q7. For each department, how many students passed with a grades under 60?

In [58]:

R9= df_inner_merge_stud1[df_inner_merge_stud1.degree <60].groupby(["DepartmentName"]).StudentId.nunique().reset_index()

R9.columns = ['Department Name','Student Count60']
R9

R91=pd.merge(R9,R1, on=['Department Name'])
R91

R91
R91['count_percent60']=(R91['Student Count60'] / R91['Student Count'])*100.0
R91 = R91[['Department Name','Student Count60','Student Count','count_percent60']]
R91


,Department Name,Student Count60,Student Count,count_percent60
0,Arts,3,137,2.189781
1,English,2,84,2.380952
2,Science,7,200,3.500000
3,Sport,1,81,1.234568


## Q8. Rate the teachers by their average student's grades (in descending order).

In [16]:
df_inner_merge_class = pd.merge(Classrooms,Courses, on="CourseId")
df_inner_merge_teach = pd.merge(df_inner_merge_class,Teachers, on="TeacherId")
R10= df_inner_merge_teach.groupby([ "FirstName","LastName"])['degree'].mean().reset_index()
      

R10 = R10.sort_values(by='degree', ascending=False)
R10.columns = [ "Teacher Name","","avg_degree"]
R10



,Teacher Name,,avg_degree
5,Darick,Hess,81.432703
9,Jordyn,Baron,80.729494
7,Heather,Metcalf,80.440242
21,Zachary,Healy,79.365867
12,Kaylea,Monzingo,79.365537
4,Daniel,Bland,78.908281
13,Keegan,Price,78.814631
3,Conor,Mckittrick,78.699492
16,Maxwell,Romkee,78.673509
20,Terran,Beaty,78.495479


## Q9. Create a dataframe showing the courses, departments they are associated with, the teacher in each course, and the number of students enrolled in the course (for each course, department and teacher show the names).

In [53]:

df1 = pd.merge(Courses, Departments, on="DepartmentId",how='left')
df2 = pd.merge(df1, Classrooms, on="CourseId",how='left')
df2 = pd.merge(df2,Teachers, on="TeacherId",how='left')
df2 = df2.groupby(['CourseId','CourseName','DepartmentName','FirstName','LastName'])['StudentId'].count().reset_index()
df2.columns = ['Course Id ','Course Name','Department Name','First Name','Last Name', 'Student Number']
df2


,Course Id,Course Name,Department Name,First Name,Last Name,Student Number
0,1,English Begginers,English,Conor,Mckittrick,32
1,2,Advanced English,English,Conor,Mckittrick,24
2,3,Proffesional English,English,Conor,Mckittrick,38
3,4,Mathematics 101,Science,Amanda,Dworkin,18
4,5,Matematics Advanced,Science,Alexander,Watts,10
5,6,Algebra,Science,Alexander,Watts,12
6,7,Geometry,Science,Alexander,Watts,14
7,8,Chemistry,Science,Jeffrey,Batzel,24
8,9,Physics,Science,Alexander,Watts,12
9,10,Biology,Science,Jordyn,Baron,36


## Q10. Create a dataframe showing the students, the number of courses they take, the average of the grades per class, and their overall average (for each student show the student name).

In [56]:
df3 = pd.merge(students, Classrooms, on='StudentId',how='left')
df5 = pd.merge(df3, Courses, on='CourseId',how='left')

df51=df5.groupby(['StudentId', 'FirstName','LastName']).CourseId.nunique().reset_index()
df51.columns = ['StudentId','First Name','Last Name', 'Count_Courses']
df51

df_eng= df5[df5.DepartmentId == 1].groupby(['StudentId'])['degree'].mean().reset_index()
df_eng.columns = ['StudentId','English']
df_eng

df55= pd.merge(df51, df_eng, on='StudentId',how='left')
df55

df_sci= df5[df5.DepartmentId == 2].groupby(['StudentId'])['degree'].mean().reset_index()
df_sci.columns = ['StudentId','Science']
df_sci

df551= pd.merge(df55, df_sci, on='StudentId',how='left')
df551

df_art= df5[df5.DepartmentId == 3].groupby(['StudentId'])['degree'].mean().reset_index()
df_art.columns = ['StudentId','Arts']
df_art

df552= pd.merge(df551, df_art, on='StudentId',how='left')
df552

df_sprt= df5[df5.DepartmentId == 4].groupby(['StudentId'])['degree'].mean().reset_index()
df_sprt.columns = ['StudentId','Sports']
df_sprt

df553= pd.merge(df552, df_sprt, on='StudentId',how='left')
df553

df56=df5.groupby(['StudentId'])['degree'].mean().reset_index()
df56.columns = ['StudentId','general_avg']
df56

df6 = pd.merge(df553, df56, on='StudentId',how='left')
df6.columns = ['StudentId','FirstName','LastName','English','Arts', 'Science', 'Sports','Count_Courses',"general_avg"]
df6







,StudentId,FirstName,LastName,English,Arts,Science,Sports,Count_Courses,general_avg
0,1,Alexandria,Heller,4,NaN,79.557508,86.677535,NaN,81.337515
1,2,Cassie,Liska,5,NaN,78.867115,79.542738,NaN,79.137364
2,3,Hayley,Frank,3,83.972245,76.204427,NaN,NaN,78.793700
3,4,Maxwell,Kotch,3,70.929801,74.157066,69.356144,NaN,71.481003
4,5,Dylan,Boehm,3,NaN,100.000000,64.494845,65.661106,76.718650
...,...,...,...,...,...,...,...,...,...
275,276,Dylan,Bannister,3,NaN,76.784735,NaN,NaN,76.784735
276,277,Bryan,Medina,6,NaN,81.298837,78.849330,NaN,80.482335
277,278,Sydney,Blas,3,NaN,72.578017,NaN,76.749775,75.359189
278,279,Samantha,Berend,1,NaN,NaN,NaN,NaN,NaN
